In [76]:
from dateutil.parser import parse
import csv, sqlite3

def fix_title(title):
    if ',' in title:
        parts = title.split(', ')
        if len(parts) == 2:
            title = parts[1] + ' ' + parts[0]
    return title

def extract_year(date_str):
    try:
        parsed_date = parse(date_str)
        return str(parsed_date.year)
    except ValueError:
        return None

con = sqlite3.connect("Movie.db")
cur = con.cursor()

cur.execute("DROP TABLE Movie1;")
cur.execute("DROP TABLE Movie2;")
cur.execute("DROP TABLE Movie3;")
cur.execute("DROP TABLE Movie4;")


cur.execute("CREATE TABLE Movie1 (budget, genres, homepage, keywords, original_language, original_title, overview, popularity, production_companies, production_countries, release_date, revenue, runtime, spoken_languages, status, tagline, title, vote_average, vote_count);")
cur.execute("CREATE TABLE Movie2 (name, rating, genre, year, released, score, votes, director, writer, star, country, budget, gross, company, runtime);")
cur.execute("CREATE TABLE Movie3 (Poster_Link, Series_Title, Released_Year, Certificate, Runtime, Genre, IMDB_Rating, Overview, Meta_score, Director, Star1, Star2, Star3, Star4, No_of_Votes, Gross);")
cur.execute("CREATE TABLE Movie4 (title, year, bechdelRating, imdbAverageRating, numVotes, runtimeMinutes, genre1, genre2, genre3);")


def load_data(csv_file, table_name, columns):
    with open(csv_file, 'r', encoding='utf-8') as fin:
        dr = csv.DictReader(fin)
        to_db = []
        for i in dr:
            fixed_row = []
            for col in columns:
                if col == 'title':
                    fixed_row.append(fix_title(i[col]))
                elif col == 'release_date':
                    fixed_row.append(extract_year(i['release_date']))
                else:
                    fixed_row.append(i[col])
            to_db.append(tuple(fixed_row))

    cur.executemany(f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES ({', '.join(['?']*len(columns))});", to_db)
    con.commit()

load_data('/content/tmdb_5000_movies.csv', 'Movie1', ["budget", "genres", "homepage", "keywords", "original_language", "original_title", "overview", "popularity", "production_companies", "production_countries", "release_date", "revenue", "runtime", "spoken_languages", "status", "tagline", "title", "vote_average", "vote_count"])
load_data('/content/movies.csv', 'Movie2', ["name", "rating", "genre", "year", "released", "score", "votes", "director", "writer", "star", "country", "budget", "gross", "company", "runtime"])
load_data('/content/imdb_top_1000.csv', 'Movie3', ["Poster_Link", "Series_Title", "Released_Year", "Certificate", "Runtime", "Genre", "IMDB_Rating", "Overview", "Meta_score", "Director", "Star1", "Star2", "Star3", "Star4", "No_of_Votes", "Gross"])
load_data('/content/Bechdel_IMDB_Merge0524.csv', 'Movie4', ["title", "year", "bechdelRating", "imdbAverageRating", "numVotes", "runtimeMinutes", "genre1", "genre2", "genre3"])

con.close()


In [ ]:
import sqlite3
import pandas as pd

# Connessione al database SQLite
con = sqlite3.connect("Movie.db")

# Funzione per visualizzare i dati di una tabella
def display_table(table_name):
    query = f"SELECT * FROM {table_name};"
    df = pd.read_sql_query(query, con)
    print(f"--- {table_name} ---")
    display(df)  # Se usi un Jupyter Notebook, altrimenti usa print(df)

# Visualizza i dati per ciascuna tabella
display_table('Movie1')
display_table('Movie2')
display_table('Movie3')
display_table('Movie4')

# Chiusura della connessione al database
con.close()

In [81]:
import sqlite3
import pandas as pd

con = sqlite3.connect("Movie.db")

query = """
SELECT *
FROM Movie4 m4
LEFT JOIN Movie3 m3 ON m4.title = m3.Series_Title AND m4.year = m3.Released_Year
LEFT JOIN Movie2 m2 ON m4.title = m2.name AND m4.year = m2.year
LEFT JOIN Movie1 m1 ON m4.title = m1.title AND m4.year = m1.release_date ;
"""

df = pd.read_sql_query(query, con)



con.close()

In [82]:
df.head()

,title,year,bechdelRating,imdbAverageRating,numVotes,runtimeMinutes,genre1,genre2,genre3,Poster_Link,...,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,Miss Jerry,1894,0,5.4,212.0,45,Romance,,,None,...,None,None,None,None,None,None,None,None,None,None
1,The Story of the Kelly Gang,1906,1,6.0,903.0,70,Action,Adventure,Biography,None,...,None,None,None,None,None,None,None,None,None,None
2,Cleopatra,1912,2,5.1,622.0,100,Drama,History,,None,...,None,None,None,None,None,None,None,None,None,None
3,A Florida Enchantment,1914,2,5.8,300.0,63,Comedy,,,None,...,None,None,None,None,None,None,None,None,None,None
4,The Birth of a Nation,1915,2,6.1,26403.0,195,Drama,History,War,None,...,None,None,None,None,None,None,None,None,None,None


In [83]:
df.to_csv("Movies_Combined.csv", index=False, encoding='utf-8-sig')

print("Dati esportati con successo in 'Movies_Combined.csv'")

Dati esportati con successo in 'Movies_Combined.csv'
